In [1]:
import argparse
from loguru import logger
import os
from os.path import join
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import bitsandbytes as bnb
from torch import nn


In [2]:
import deepspeed


[2025-02-17 11:48:01,434] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/root/miniconda3/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/root/minicond

In [3]:
deepspeed.ops.op_builder.CPUAdamBuilder().load()

Using /root/.cache/torch_extensions/py312_cu121 as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/py312_cu121/cpu_adam/build.ninja...
Building extension module cpu_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.
Time to load cpu_adam op: 2.545668601989746 seconds


Loading extension module cpu_adam...


<module 'cpu_adam' from '/root/.cache/torch_extensions/py312_cu121/cpu_adam/cpu_adam.so'>

In [4]:
from component.collator import PretrainCollator, SFTDataCollator
from component.argument import CustomizedArguments
from component.template import template_dict
from component.dataset import (
    UnifiedSFTDataset,
    ChatGLM2SFTDataset,
    ChatGLM3SFTDataset,
    UnifiedDPODataset,
    UnifiedSFTToolsDataset
)

Reading data: 51it [00:00, 81303.50it/s]


In [5]:
from transformers import (
    set_seed,
    HfArgumentParser,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoConfig,
    BitsAndBytesConfig,
    Trainer,
    AddedToken
)

In [6]:
import importlib
if importlib.util.find_spec('unsloth') is not None:
    from unsloth import FastLanguageModel


In [7]:
from datasets import load_dataset, concatenate_datasets
import datasets
from itertools import chain
from tqdm import tqdm
import json
from trl import DPOTrainer, get_kbit_device_map
import torch.nn as nnF


In [8]:
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['PATH'] = os.environ["PATH"] + ":/opt/conda/bin/ninja"


In [9]:
# def setup_everything():

# '''参数解析，通过解析train_args_file 得到参数'''
parser = argparse.ArgumentParser()
# parser.add_argument("--train_args_file", type=str, default='train_args/pretrain/full/bloom-1b1-pretrain-full.json', help="")
parser.add_argument("--train_args_file", type=str, default='train_args/sft/full/qwen2-1.5b-sft-full.json', help="")
parser.add_argument("--local_rank", type=int, help="")
args = parser.parse_args()
train_args_file = args.train_args_file
# 读取训练的参数配置，把两种参数合并在一起
# CustomizedArguments：这通常是一个用户自定义的参数类，用于包含除了标准训练参数之外，用户想要添加的任何特定参数
# TrainingArguments：这是 Hugging Face Transformers 库中预定义的一个类，包含了一系列用于模型训练的标准参数，如学习率、训练轮数（epochs）、批大小（batch size）、输出目录
parser = HfArgumentParser((CustomizedArguments, TrainingArguments))

# 解析得到自定义参数，以及自带参数
args, training_args = parser.parse_json_file(json_file=train_args_file)



# # 创建输出目录
# if not os.path.exists(training_args.output_dir):
#     os.makedirs(training_args.output_dir)
# logger.add(join(training_args.output_dir, 'train.log'))
# logger.info("args:{}".format(args))
# logger.info("train_args:{}".format(training_args))
# # 加载训练配置文件
# with open(train_args_file, "r") as f:
#     train_args = json.load(f)
# # 保存训练参数到输出目录
# with open(join(training_args.output_dir, 'train_args.json'), "w") as f:
#     json.dump(train_args, f, indent=4)
# # 设置随机种子
# set_seed(training_args.seed)

# # check some setting
# assert args.task_type in ['pretrain', 'sft', 'dpo'], "task_type should be in ['pretrain', 'sft', 'dpo']"
# assert args.train_mode in ['full', 'lora', 'qlora'], "task_type should be in ['full', 'lora', 'qlora']"
# assert sum([training_args.fp16, training_args.bf16]) == 1, "only one of fp16 and bf16 can be True"
# # assert not (args.task_type == 'dpo' and args.use_unsloth), 'We have not tested Unsloth during DPO yet. Please set use_unsloth=False when task_type=dpo'



usage: ipykernel_launcher.py [-h] [--train_args_file TRAIN_ARGS_FILE]
                             [--local_rank LOCAL_RANK]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-5a3c6a78-943f-4891-96d2-e896711288cd.json


SystemExit: 2

/root/miniconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:
import torch
def supports_flash_attention(device_id: int):
    """Check if a GPU supports FlashAttention."""
    major, minor = torch.cuda.get_device_capability(device_id)
    
    # Check if the GPU architecture is Ampere (SM 8.x) or newer (SM 9.0)
    is_sm8x = major == 8 and minor >= 0
    is_sm90 = major == 9 and minor == 0
    return is_sm8x or is_sm90
print(supports_flash_attention(0))

True


In [12]:
class ModelArguments:
    def __init__(self, model_name_or_path):
        self.model_name_or_path = "/root/autodl-tmp/full_or_lora_model/fine-tuning-full/Qwen2-1.5B"

# 使用示例
args = ModelArguments('my_model_name_or_path')

config = AutoConfig.from_pretrained(args.model_name_or_path, trust_remote_code=True)
# 加载tokenzier
tokenizer = AutoTokenizer.from_pretrained(
    args.model_name_or_path,
    #这个参数允许从远程源（如 Hugging Face Models Hub）加载和执行可能包含自定义代码（如特殊分词逻辑）的模型。这在某些情况下是必要的，因为一些模型可能包含特定的预处理或后处理步骤
    trust_remote_code=True,
    # 这个参数决定了是否应该使用基于 Rust 的 tokenizers 库（通常称为“fast”分词器）来加速分词过程
    # llama不支持fast
    use_fast=False if config.model_type == 'llama' or config.model_type == 'internlm2' else True
)

In [13]:
print(tokenizer.__class__.__name__)
# print(tokenizer.eod_id)
print(tokenizer.pad_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.vocab_size)

Qwen2TokenizerFast
151643
151643
151643


In [14]:
quantization_config = None

torch_dtype = torch.bfloat16

model_kwargs = dict(
    trust_remote_code=True, # 加载远程代码
    attn_implementation="flash_attention_2", # 注意力机制，flash_attention 可加速，目前支持Nvidia GPU显卡（如H100、A100、RTX X090、T4) ，如果不支持的话，这一行注释掉即可
    torch_dtype=torch_dtype, # 模型精度
    # gradient_checkpointing取值是false
    # 如果显存捉襟见肘，可以开启。以时间换空间，模型不缓存激活状态，会进行两次forward计算，以节省显存
    use_cache=True,
    device_map=None, # 模型配置
    quantization_config=quantization_config, # 量化配置
)

model = AutoModelForCausalLM.from_pretrained(args.model_name_or_path, **model_kwargs)



You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [15]:
model.config.to_dict()

{'vocab_size': 151936,
 'max_position_embeddings': 131072,
 'hidden_size': 1536,
 'intermediate_size': 8960,
 'num_hidden_layers': 28,
 'num_attention_heads': 12,
 'use_sliding_window': False,
 'sliding_window': None,
 'max_window_layers': 28,
 'num_key_value_heads': 2,
 'hidden_act': 'silu',
 'initializer_range': 0.02,
 'rms_norm_eps': 1e-06,
 'use_cache': True,
 'rope_theta': 1000000.0,
 'rope_scaling': None,
 'attention_dropout': 0.0,
 'return_dict': True,
 'output_hidden_states': False,
 'output_attentions': False,
 'torchscript': False,
 'torch_dtype': 'bfloat16',
 'use_bfloat16': False,
 'tf_legacy_loss': False,
 'pruned_heads': {},
 'tie_word_embeddings': True,
 'chunk_size_feed_forward': 0,
 'is_encoder_decoder': False,
 'is_decoder': False,
 'cross_attention_hidden_size': None,
 'add_cross_attention': False,
 'tie_encoder_decoder': False,
 'max_length': 20,
 'min_length': 0,
 'do_sample': False,
 'early_stopping': False,
 'num_beams': 1,
 'num_beam_groups': 1,
 'diversity_pena

In [18]:
data_path = "./data/summary/"
for root, dir_names, file_names in os.walk(data_path):
    print(root, dir_names, file_names)

./data/summary/ [] ['sum_train_2048.jsonl', 'summary_test.jsonl']


In [19]:
file = "./data/summary/sum_train_2048.jsonl"
file_name = os.path.basename(file)
print(file_name)

sum_train_2048.jsonl


In [20]:
from datasets import load_dataset, concatenate_datasets

tmp_cache_path = "./data/summary/cache/sum_train_2048/tmp3"
raw_dataset = load_dataset("json", data_files=file, cache_dir=tmp_cache_path, keep_in_memory=False)
print(raw_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['conversation', 'conversation_id', 'category', 'dataset'],
        num_rows: 1033
    })
})


In [1]:
def tokenize_function(examples):
    # print("examples=", examples)
    output = tokenizer(examples["conversation"])
    # output = {'input_ids': output.input_ids}
    print("output=", output)
    return output
        
tokenized_dataset = raw_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=10,
    remove_columns="conversation",
    load_from_cache_file=True,
    keep_in_memory=False,
    cache_file_names={k: os.path.join(tmp_cache_path, 'tokenized.arrow') for k in raw_dataset},
    desc="Running tokenizer on dataset",
)

NameError: name 'raw_dataset' is not defined

In [42]:
for example in raw_dataset['train']:
    print(example)
    break

{'conversation': [{'human': '“好雨知时节，当春乃发生。”描写了怎样的场景？', 'assistant': '在春天里下雨的场景。'}], 'conversation_id': None, 'category': None, 'dataset': None}


In [24]:
system_format = '<|im_start|>system\n{content}<|im_end|>\n'
system = "You are a helpful assistant."
system_text = system_format.format(content=system)
print(system_text)

a = tokenizer.encode(system_text, add_special_tokens=False) 
print(a)

<|im_start|>system
You are a helpful assistant.<|im_end|>

[151644, 8948, 198, 2610, 525, 264, 10950, 17847, 13, 151645, 198]


In [36]:
print(tokenizer.encode('<|im_start|>', add_special_tokens=False))
print(tokenizer.encode('<|im_end|>', add_special_tokens=False))
print(tokenizer.encode('\n', add_special_tokens=False))
print(tokenizer.encode('<|endoftext|>', add_special_tokens=False))

print(tokenizer.decode(198))
print(tokenizer.eos_token)
print(tokenizer.pad_token_id)

[151644]
[151645]
[198]
[151643]


<|endoftext|>
151643


In [35]:
human = "“好雨知时节，当春乃发生。”描写了怎样的场景？"
user_format = '<|im_start|>user\n{content}<|im_end|>\n<|im_start|>assistant\n'
human = user_format.format(content=human, stop_token=tokenizer.eos_token)
print("human=",human)

human= <|im_start|>user
“好雨知时节，当春乃发生。”描写了怎样的场景？<|im_end|>
<|im_start|>assistant



In [3]:
import pathlib
input_file = "/root/fine-tuning/data/summary/summary_test.jsonl"

input_file = pathlib.Path(input_file).stem
print(input_file)

summary_test


In [6]:
import random
prefix_list = ['给出下面这段文本的摘要，并保留原文中出现的要点序号：']
prefix = random.choice(prefix_list)
print(prefix)

给出下面这段文本的摘要，并保留原文中出现的要点序号：
